In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


In [13]:


# Chemins vers les données
train_dir = 'PlantVillage_split/train'
val_dir = 'PlantVillage_split/val'
test_dir = 'PlantVillage_split\test'

# Préparation des données avec augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Dimension des images
    batch_size=32,
    class_mode='categorical'  # Pour plusieurs classes
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Construction du modèle
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Nombre de classes
])

# Compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entraînement
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Sauvegarde du modèle
model.save('plant_disease_model.h5')




Found 14440 images belonging to 15 classes.
Found 3097 images belonging to 15 classes.


C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module,

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd

# Charger le modèle
model = load_model('plant_disease_model.h5')

# Charger les informations des maladies à partir du fichier CSV
csv_path = "plant_disease_info.csv"  # Assurez-vous que ce fichier existe
disease_info = pd.read_csv(csv_path)

# Charger une image pour la prédiction
img_path = 'yellow.JPG'  # Remplacez par le chemin de l'image
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Prédiction
predictions = model.predict(img_array)
class_indices = train_generator.class_indices  # Indices des classes utilisés à l'entraînement
class_names = list(class_indices.keys())       # Liste des noms des classes
predicted_class = class_names[np.argmax(predictions)]  # Classe prédite

# Rechercher les informations correspondantes à la maladie prédite
disease_row = disease_info[disease_info["Maladie"] == predicted_class]

if not disease_row.empty:
    description = disease_row.iloc[0]["Description"]
    solution = disease_row.iloc[0]["Solution"]
    print(f"La plante est : {predicted_class}")
  
    print(f"Description : {description}")
    print(f"Solution : {solution}")
else:
    print(f"La plante est : {predicted_class}")
    print("Aucune information supplémentaire n'est disponible pour cette maladie.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
La plante est : Tomato__Tomato_YellowLeaf__Curl_Virus
Description : Feuilles jaunes recourbées causées par un virus transmis par les aleurodes.
Solution : Contrôler les aleurodes avec des insecticides et utiliser des variétés résistantes.


In [15]:
import pandas as pd

# Liste complète des maladies, descriptions et solutions
data = [
    {"Maladie": "Pepper_bell__Bacterial_spot", 
     "Description": "Tâches sombres sur les feuilles et fruits causées par des bactéries.", 
     "Solution": "Utiliser des semences résistantes, éviter une humidité excessive et appliquer des traitements bactéricides."},
    {"Maladie": "Pepper_bell__healthy", 
     "Description": "Aucune maladie détectée. La plante est en bonne santé.", 
     "Solution": "Maintenir de bonnes pratiques agricoles pour éviter les maladies."},
    {"Maladie": "Potato__Early_blight", 
     "Description": "Taches brunes irrégulières avec des cercles concentriques sur les feuilles.", 
     "Solution": "Éviter l'arrosage excessif et appliquer des fongicides adaptés."},
    {"Maladie": "Potato__healthy", 
     "Description": "Aucune maladie détectée. La plante est en bonne santé.", 
     "Solution": "Maintenir de bonnes pratiques agricoles pour éviter les maladies."},
    {"Maladie": "Potato__Late_blight", 
     "Description": "Taches noires qui s'étendent rapidement sur les feuilles et les tubercules.", 
     "Solution": "Utiliser des variétés résistantes et des fongicides préventifs."},
    {"Maladie": "Tomato_Target_Spot", 
     "Description": "Taches cibles avec un centre clair entouré d'un bord sombre sur les feuilles et fruits.", 
     "Solution": "Appliquer des fongicides et éviter une humidité prolongée."},
    {"Maladie": "Tomato_Tomato_mosaic_virus", 
     "Description": "Feuilles tachetées et déformées causées par le virus de la mosaïque.", 
     "Solution": "Éliminer les plantes infectées et désinfecter les outils de jardinage."},
    {"Maladie": "Tomato__Tomato_YellowLeaf__Curl_Virus", 
     "Description": "Feuilles jaunes recourbées causées par un virus transmis par les aleurodes.", 
     "Solution": "Contrôler les aleurodes avec des insecticides et utiliser des variétés résistantes."},
    {"Maladie": "Tomato_Bacterial_spot", 
     "Description": "Petites taches sombres sur les feuilles et les fruits des tomates causées par des bactéries.", 
     "Solution": "Pratiquer la rotation des cultures et utiliser des produits bactéricides."},
    {"Maladie": "Tomato_Early_blight", 
     "Description": "Taches sombres entourées de cercles concentriques sur les feuilles.", 
     "Solution": "Éliminer les feuilles infectées et appliquer des fongicides."},
    {"Maladie": "Tomato_healthy", 
     "Description": "Aucune maladie détectée. La plante est en bonne santé.", 
     "Solution": "Continuer les bonnes pratiques pour éviter les maladies."},
    {"Maladie": "Tomato_Late_blight", 
     "Description": "Taches d'eau qui deviennent noires sur les feuilles et fruits.", 
     "Solution": "Éviter les conditions humides prolongées et appliquer des fongicides."},
    {"Maladie": "Tomato_Leaf_Mold", 
     "Description": "Moisissure jaune ou brune sur la face inférieure des feuilles.", 
     "Solution": "Améliorer la ventilation et appliquer des traitements anti-moisissures."},
    {"Maladie": "Tomato_Septoria_leaf_spot", 
     "Description": "Taches rondes et sombres avec un centre clair sur les feuilles inférieures.", 
     "Solution": "Enlever les feuilles affectées et appliquer des fongicides spécifiques."},
    {"Maladie": "Tomato_Spider_mites_Two_spotted_spider_mite", 
     "Description": "Feuilles jaunissantes avec présence de toiles d'araignée fines.", 
     "Solution": "Utiliser des acaricides et augmenter l'humidité pour réduire les infestations."}
]

# Création du DataFrame
df = pd.DataFrame(data)

# Sauvegarde dans un fichier CSV
df.to_csv("plant_disease_info.csv", index=False, encoding='utf-8')

print("Le fichier CSV contenant toutes les maladies a été créé avec succès !")


Le fichier CSV contenant toutes les maladies a été créé avec succès !
